In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math

from IPython.display import clear_output

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [3]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 150

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]

trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]
fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]


fileNamesDF = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])
    
fileNamesDF = fileNamesDF['filename']

In [4]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo = allInfo[allInfo['new_filename'].isin(fileNamesDF)]

In [5]:
featuresDF = pd.DataFrame(allInfo['new_filename'])
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']

In [6]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [7]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF, bands

    return imageDF.mean().values

In [8]:
featuresDF.index

Int64Index([   142,    149,    213,    308,    414,    511,    697,    773,
               856,    867,
            ...
            101587, 101629, 101725, 102085, 102113, 102402, 102565, 102862,
            103030, 103049],
           dtype='int64', length=2254)

In [10]:
imageFeaturesDF = pd.DataFrame(index=featuresDF.index,columns=['features']) 
print(imageFeaturesDF.index)
for row in tqdm(list(featuresDF.index)):
    imageFeatures = featurizeImage(featuresDF.loc[row]['new_filename'])
    imageFeaturesDF.loc[row]['features'] = imageFeatures


Int64Index([   142,    149,    213,    308,    414,    511,    697,    773,
               856,    867,
            ...
            101587, 101629, 101725, 102085, 102113, 102402, 102565, 102862,
            103030, 103049],
           dtype='int64', length=2254)


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/adrien/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/adrien/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/adrien/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [11]:
imageFeaturesDF.head()

,features
142,"[0.4953544452320413, 0.4249800919821063, 0.388..."
149,"[0.5563707686151453, 0.6178211379666099, 0.667..."
213,"[0.47377666637491667, 0.4798133166152557, 0.50..."
308,"[0.6521940708244118, 0.5543948635097269, 0.542..."
414,"[0.6056229039301642, 0.6927737172437644, 0.679..."


In [12]:
imageFeaturesDF.to_csv('image_features_extracted3.csv')

In [13]:
featuresDF['imgFeatures'] = imageFeaturesDF['features']

In [14]:
featuresDF.head()

,new_filename,pixelsx,pixelsy,imgFeatures
142,100829.jpg,3672.0,5004.0,"[0.4953544452320413, 0.4249800919821063, 0.388..."
149,16019.jpg,4806.0,3732.0,"[0.5563707686151453, 0.6178211379666099, 0.667..."
213,100182.jpg,4740.0,3474.0,"[0.47377666637491667, 0.4798133166152557, 0.50..."
308,12455.jpg,4722.0,3102.0,"[0.6521940708244118, 0.5543948635097269, 0.542..."
414,16347.jpg,4092.0,3294.0,"[0.6056229039301642, 0.6927737172437644, 0.679..."


In [20]:
def compareImages(img1, img2):
    
    diffx = np.abs(img1['pixelsx']-img2['pixelsx']) / (img1['pixelsx'] + img2['pixelsx'])
    diffy = np.abs(img1['pixelsy']-img2['pixelsy']) / (img1['pixelsy'] + img2['pixelsy'])
    
    diffSize = (diffx + diffy)/2
    
    img1Features = img1['imgFeatures']
    img2Features = img2['imgFeatures']
    
    imgFeatureScore = 0
    
    if (len(img1Features) == len(img2Features)):
        imgFeatureDiffs = np.abs(img1Features - img2Features)
        imgFeatureScore = sum(imgFeatureDiffs)
        score = (diffSize+imgFeatureScore)/2
    else:
        score = (diffSize+imgFeatureScore)/2

   
    return 1-score

In [21]:
def generatePrediction(score,threshold):
    if score > threshold:
        return True
    else:
        return False

def generatePredictions(scores,threshold):
    predictions = []
    for curScore in tqdm(scores,leave=False):
        predictions.append(generatePrediction(curScore,threshold))

    return predictions

In [22]:
def trainThreshold(featuresTrain,maxNumIterations = 100,initialProb = 0.5, increaseRate = 0.001):
    
    probThreshold = initialProb
    prevAccuracy = 0

    for i in tqdm(range(maxNumIterations),leave=False):
        # the score represents how similar two images are

        scores = []
        iloc1 = featuresTrain.sample(frac=0.5).index
        iloc2 = featuresTrain.sample(frac=0.5).index
        # print(len(iloc2))
        # print(nTrials)

        for i, loc in tqdm(enumerate(iloc1),leave=False):
            scores.append(compareImages(featuresTrain.loc[loc],featuresTrain.loc[iloc2[i]]))

        filenames1 = featuresTrain.loc[iloc1]['new_filename'].values
        filenames2 = featuresTrain.loc[iloc2]['new_filename'].values


        truth = trainInfo[trainInfo['filename'].isin(filenames1)]['artist'].values == trainInfo[trainInfo['filename'].isin(filenames2)]['artist'].values
        predictions = generatePredictions(scores,probThreshold)

        accuracy = (predictions == truth).sum()/len(predictions == truth)
        if (accuracy < prevAccuracy):
            print("peak accuracy")
            return probThreshold, prevAccuracy
        
        probThreshold += increaseRate
    
    return probThreshold, accuracy

In [23]:
def computeDevAcc(featuresTrain,probability):

    # the score represents how similar two images are
    scores = []
    iloc1 = featuresTrain.sample(frac=0.5).index
    iloc2 = featuresTrain.sample(frac=0.5).index

    # print(len(iloc2))
    # print(nTrials)

    for i, loc in tqdm(enumerate(iloc1),leave=False):
        scores.append(compareImages(featuresTrain.loc[loc],featuresTrain.loc[iloc2[i]]))

    filenames1 = featuresTrain.loc[iloc1]['new_filename'].values
    filenames2 = featuresTrain.loc[iloc2]['new_filename'].values


    truth = trainInfo[trainInfo['filename'].isin(filenames1)]['artist'].values == trainInfo[trainInfo['filename'].isin(filenames2)]['artist'].values
    predictions = generatePredictions(scores,probability)

    accuracy = (predictions == truth).sum()/len(predictions == truth)
    return accuracy

In [ ]:
nTrainFraction = 0.6
nDevFraction = 0.4

thresholds = []
devAccuracies = []
numberSamples = 10
for i in tqdm(range(numberSamples)):
    featuresTrain = featuresDF.sample(frac=nTrainFraction)
    featuresDev = featuresDF.drop(featuresTrain.index)
#     featuresTest = featuresDF.drop(featuresDev.index).drop(featuresTrain.index).sample(n=nTest)
    learnedThreshold, trainAccuracy = trainThreshold(featuresTrain,50,0.5,0.01)
    thresholds.append((learnedThreshold,trainAccuracy))
    meanDevAccuracy = 0
    accuraciesDev = []
    for i in range(10):
        accuraciesDev.append(computeDevAcc(featuresDev,learnedThreshold))
    meanDevAccuracy = np.mean(accuraciesDev)
    devAccuracies.append(meanDevAccuracy)
    
print(thresholds)
print(devAccuracies)